In [1]:
#Models
from models.oven_db import Oven
from models.nut_batch_db import NutBatch, NutType
from models.real_butch_data_db import RealTimeBatchData

#Services
from services.oven_service import OvenService
from services.nut_batch_service import NutBatchService
from services.real_batch_service import RealTimeBatchDataService

In [2]:
oven_id = "67f3f1d0f93c97b2aec11525"
oven_service = OvenService()
nut_batch_service = NutBatchService()
real_batch_service = RealTimeBatchDataService()

In [ ]:
#Crear un horno
oven = Oven("Horno 2", "Chilecito")
oven_service.create_oven(oven)

In [7]:
#Obtener todos los hornos disponibles
result = oven_service.list_ovens()
print(result)

[{'id': '67f3f1d0f93c97b2aec11525', 'name': 'Horno 2'}]


In [6]:
#Obtener un horno
result = oven_service.get_oven("67f3f0fff93c97b2aec11524")
print(result)

None


In [5]:
#Eliminar un horno
oven_service.delete_oven("67f3f0fff93c97b2aec11524")

DeleteResult({'n': 1, 'electionId': ObjectId('7fffffff000000000000002a'), 'opTime': {'ts': Timestamp(1744047277, 8), 't': 42}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1744047277, 8), 'signature': {'hash': b'(\xc1\x19N+&s\x9a\xb6\xd4\xf6C\x1ccS\x0b+O\xb5\xee', 'keyId': 7448678712490328069}}, 'operationTime': Timestamp(1744047277, 8)}, acknowledged=True)

In [4]:
#Crear un lote de nueces previo al secado
nut_batch = NutBatch(
    nut_type=NutType.HALF,
    initial_weight=50.36,
    initial_humidity=36.25,
    target_humidity=20.23,
    oven_id="67f3f1d0f93c97b2aec11525"
)
result = nut_batch_service.create_batch(nut_batch)

Excepcion Existe una batch en este horno sin completar, por favor finaliza la accion de secado


In [5]:
#Obtener todos los lotes
batchs = nut_batch_service.list_batches()
print(batchs)

[{'_id': ObjectId('67f51e064126a6e6cba17bc1'), 'nut_type': 'HALF', 'start_date': datetime.datetime(2025, 4, 8, 10, 0, 53, 789000), 'end_date': None, 'initial_weight': 50.36, 'initial_humidity': 36.25, 'target_humidity': 20.23, 'oven_id': '67f3f1d0f93c97b2aec11525', 'drying_history': [], 'status': 'IN_PROGRESS'}]


In [ ]:
#Insertar datos temporales en redis sobre los lotes
batch = nut_batch_service.get_actual_batch_by_oven(oven_id)
data = RealTimeBatchData(
    batch_id=str(batch["_id"]),
    temperature=12,
    humidity=17,
    weight=10,
    gas_consumption=18,
)

real_batch_service.update_data(data)



In [4]:
#Obtener todos los registros temporales para un lote
data = real_batch_service.get_all_data("67f51e064126a6e6cba17bc1")
print(data)

[{'batch_id': '67f51e064126a6e6cba17bc1', 'temperature': 55, 'humidity': 31, 'weight': 13, 'gas_consumption': 12, 'timestamp': '2025-04-08T23:20:04.904431'}, {'batch_id': '67f51e064126a6e6cba17bc1', 'temperature': 53, 'humidity': 30, 'weight': 12, 'gas_consumption': 15, 'timestamp': '2025-04-08T23:20:29.736722'}, {'batch_id': '67f51e064126a6e6cba17bc1', 'temperature': 12, 'humidity': 17, 'weight': 10, 'gas_consumption': 18, 'timestamp': '2025-04-08T23:27:17.408712'}, {'batch_id': '67f51e064126a6e6cba17bc1', 'temperature': 12, 'humidity': 17, 'weight': 10, 'gas_consumption': 18, 'timestamp': '2025-04-08T23:27:45.257424'}, {'batch_id': '67f51e064126a6e6cba17bc1', 'temperature': 12, 'humidity': 17, 'weight': 10, 'gas_consumption': 18, 'timestamp': '2025-04-09T00:06:34.867247'}, {'batch_id': '67f51e064126a6e6cba17bc1', 'temperature': 12, 'humidity': 17, 'weight': 10, 'gas_consumption': 18, 'timestamp': '2025-04-09T00:08:16.035295'}]


In [3]:
#Finalizacion de proceso de secado
result = nut_batch_service.add_drying_data(oven_id)
print(result)

20.23 17
UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff000000000000002b'), 'opTime': {'ts': Timestamp(1744169538, 4), 't': 43}, 'nModified': 0, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1744169538, 4), 'signature': {'hash': b'\x1e\x8e%,\xbb\xadt\x93?\\\xc6i\x8e\xfe\xae%\xfa;u\xeb', 'keyId': 7448678712490328069}}, 'operationTime': Timestamp(1744169538, 4), 'updatedExisting': True}, acknowledged=True)
